# Entrenamiento con reducción de dimensión con PCA
Atributos del tipo likert se tratan como categóricos ordinales, se aplica one-hot-encoding a los datos categóricos y se reduce la dimensión del conjunto por medio de PCA.  Posteriormente se entrena:

1. Un modelo por regresión logística
2. Un modelo por máquinas de soporte vectorial

En cada caso se selecciona un mejor modelo por validación cruzada.

Iniciamos por cargar la base de datos (se remueven aquellos renglones en los que hay valores desconocidos).

In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

dF_pca = pd.read_csv("sales_data.csv").dropna()
dF_pca.head(5)

,flag,gender,education,house_val,age,online,customer_psy,marriage,child,occupation,mortgage,house_owner,region,car_prob,fam_income
5,Y,F,3. Bach,248694,6upto65,Y,B,Married,N,Professional,2Med,Owner,West,1,G
7,N,F,3. Bach,416925,5upto55,Y,C,Married,Y,Professional,1Low,Owner,South,2,I
11,Y,F,3. Bach,245686,4upto45,N,F,Married,U,Blue Collar,1Low,Owner,South,3,E
12,Y,F,2. Some College,360587,5upto55,Y,C,Married,Y,Professional,3High,Owner,Midwest,1,J
15,Y,M,0. lessHS,162884,1_Unk,Y,G,Married,Y,Professional,1Low,Renter,South,7,C


## Preprocesamiento
Los valores de las columnas fam_income y car_prob (que son de tipo categórico ordinal) se convierten en númericos, 
bajo el supuesto de que valores más grandes son mejores. Se re-escalan las columnas house_val, 
fam_income y car_prob a $[0,1]$. 

In [44]:
# Rescale house_val
m, M = dF_pca.house_val.min(), dF_pca.house_val.max()
dF_pca['house_val'] = (dF_pca.house_val - m) / (M - m)

# Rescale car_prob
dF_pca['car_prob'] = dF_pca.car_prob / 10

# Change categorical ordinal values to numbers
for i in range(0,len(dF_pca)):
    if dF_pca.iloc[i,14] == 'U': dF_pca.iloc[i,14] = 0
    if dF_pca.iloc[i,14] == 'A': dF_pca.iloc[i,14] = 0
    if dF_pca.iloc[i,14] == 'B': dF_pca.iloc[i,14] = 1
    if dF_pca.iloc[i,14] == 'C': dF_pca.iloc[i,14] = 2
    if dF_pca.iloc[i,14] == 'D': dF_pca.iloc[i,14] = 3
    if dF_pca.iloc[i,14] == 'E': dF_pca.iloc[i,14] = 4
    if dF_pca.iloc[i,14] == 'F': dF_pca.iloc[i,14] = 5
    if dF_pca.iloc[i,14] == 'G': dF_pca.iloc[i,14] = 6
    if dF_pca.iloc[i,14] == 'H': dF_pca.iloc[i,14] = 7
    if dF_pca.iloc[i,14] == 'I': dF_pca.iloc[i,14] = 8
    if dF_pca.iloc[i,14] == 'J': dF_pca.iloc[i,14] = 9
    if dF_pca.iloc[i,14] == 'K': dF_pca.iloc[i,14] = 10
    if dF_pca.iloc[i,14] == 'L': dF_pca.iloc[i,14] = 11
dF_pca.head(5)

# Rescale fam_income
dF_pca['fam_income'] = dF_pca.fam_income / 11

Aplicamos one-hot-encoding para todos los valores que aún no son númericos. 

In [60]:
# Encoding
dF_pca = pd.get_dummies(dF_pca)

# Separate response column and atributes matrix
y_pca = dF_pca['flag_Y'].values
X_pre = dF_pca.drop(['flag_N','flag_Y'], axis =1).values
print("Matriz de aributos tiene dimensiones: ",  len(X_pre), "x",len(X_pre[0]))

Matriz de aributos tiene dimensiones:  23558 x 51


## PCA
Aplicamos Análisis de Componentes Principales (PCA), reteniendo aquellas columnas que explican al menos el 85% de la varianza

In [96]:
# Import PCA module and instantiate a PCA object
from sklearn.decomposition import PCA
pca = PCA(.85)

# Apply pca to atributes matrix 
pca.fit(X_pre)
var_ratios = pca.explained_variance_ratio_
print("Reducción de ", len(X_pre[0]), " a ", len(var_ratios), "dimensiones (atributos)")
print("Porcentages de varianza explicada por cada una de las 23 dimensiones conservadas:\n ", var_ratios*100)

# Reduced atributes matrix
X_pca = pca.transform(X_pre)

Reducción de  51  a  20 dimensiones (atributos)
Porcentages de varianza explicada por cada una de las 23 dimensiones conservadas:
  [12.89199597  8.32043749  7.13981809  5.46342494  4.92277369  4.72399807
  4.43113687  4.00147842  3.75092964  3.55956291  3.25409463  3.06330768
  2.92005829  2.79515602  2.77087799  2.36219632  2.34734231  2.27746684
  2.19785328  2.00314455]


Logramos reducir las dimensiones de la matriz de atributos de 51 columnas a 20 columnas. X_pca es la matriz resultante de recodificar la matriz de atributos original después de aplicar PCA. Ahora pasamos a entrenar los modelos propuestos.

# Regresión logística
Con X_pca y y_pca ahora obtenemos un modelo por regresión logística. La elección del modelo se hace por 10-fold cross validation

In [98]:
# Import scikit-learn modules and generate instance of regression model
from sklearn.linear_model import LogisticRegressionCV
logistic_reg = LogisticRegressionCV(cv = 10, random_state = 1, max_iter = 150)

# Train model on X and y, and automatically select best after cross-validation
logreg_model = logistic_reg.fit(X_pca, y_pca)

In [100]:
# Details of the model
print("Coeficientes del modelo: \n", logreg_model.coef_)
print("Score sobre el conjunto de datos completo:", logreg_model.score(X_pca,y_pca)*100)

# Quality of the model
y_pca_pred = logreg_model.predict(X_pca)
print("Cualidades de la clasificación:\n", classification_report(y_pca, y_pca_pred))
print("Matriz de confusión:\n", confusion_matrix(y_pca, y_pca_pred,labels = range(2)) )

Coeficientes del modelo: 
 [[-0.90101765 -0.33823284  0.42624811  0.52350282 -0.16222863  0.25723397
   0.33603964  0.07502424  0.06074424 -0.10480025  0.20623918 -0.01608479
  -0.0745244   0.11378817 -0.03765142  0.12189565  0.19245644  0.02074486
  -0.27067891 -0.05363118]]
Score sobre el conjunto de datos completo: 69.23762628406486
Cualidades de la clasificación:
               precision    recall  f1-score   support

           0       0.66      0.59      0.62     10219
           1       0.71      0.77      0.74     13339

   micro avg       0.69      0.69      0.69     23558
   macro avg       0.69      0.68      0.68     23558
weighted avg       0.69      0.69      0.69     23558

Matriz de confusión:
 [[ 6031  4188]
 [ 3059 10280]]


# Máquinas de soporte vectorial (SVM)
Con X_pca y y_pca obtenemos un modelo por SVMs. En la selección del modelo evaluamos modelos con diferentes coeficientes de regularización C y gama, y evaluamos con 5fold cross validation

In [80]:
# Import scikit-learn modules
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Split the data
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y_pca, test_size=0.30, random_state=1)

# SLOW!!!
# Find best SVM model through 5fold cv and grid search, radial kernel
grid = {'C': [1, 10, 100], 'gamma': [0.01, 0.05, 0.1], } #{'C': [1, 10, 100], 'gamma': [0.005, 0.01, 0.1], }
svm_model = GridSearchCV(SVC(kernel='rbf'), grid, cv=2, iid=False)

svm_model = svm_model.fit(X_pca_train, y_pca_train)
print("Best estimator found by grid search:")

Best estimator found by grid search:


In [81]:
print(svm_model.best_estimator_)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
